В Bitcoin используется криптографический алгоритм `ECDSA`, в нём всё строится на группе точек эллиптических кривых.
Существуют некоторые особые эллиптические кривые, каждая из которых названа по своему.
Есть эллиптическая кривая `secp256k1`, вот её (а точнее её параметры) используют в частности в Bitcoin.

http://www.secg.org/SEC2-Ver-1.0.pdf 15 страница, 2.7.1 - рекомендуемые параметры (base point, etc.)

Что алгоритм `ECDSA`, что эллиптическая кривая `secp256k1` были известны, использовались и до Bitcoin.

С чего нужно начать? С генерации приватного ключа. Что представляет собой приватный ключ? 256 бит данных (32 байта).
Получить свой приватный ключ можно 256 раз подкинув монетку (орёл - 0, решка - 1).

In [2]:
import random

priv_bin_key = '0b' + ''.join([str(random.randint(0, 1)) for _ in range(0, 256)])

In [3]:
priv_bin_key

'0b0010101001100101111001010101111000011101110001011011111010010101000110110110111011010001111100111111110000110101110010010001110000010100111000110111111111011100000000001100001100100111110000000110001101100010101010011110010111001001000100110101011011110000'

`0b` - это префикс, чтобы python понимал, что в строке бинарные данные. Всё остальное - "Орел" или "Решка".
И по сути уже это и является приватным ключом.

Логично, что мы можем представить это как десятичное число:

In [4]:
int(priv_bin_key, 2)

19177174231323610692260439159204199819966576577409867627808962287955388618480

Вот это большое число и есть наш приватный ключ в понятном любому человеку формате. В принципе, мы можем придумать сразу его. Только нюанс - число должно быть меньше `1.158 * 10^77` и больше 0. Т.е. число 1 тоже может являться приватным ключом, только небезопасным 😊

Далее мы можем представить его как шестнадцатиричные данные:

In [7]:
priv_hex_key = hex(int(priv_bin_key, 2))

priv_hex_key

'0x2a65e55e1dc5be951b6ed1f3fc35c91c14e37fdc00c327c06362a9e5c91356f0'

Функция hex добавлять приставку 0x в начало 16-ричной строки. Мы её удалим, она нам будет мешать.

Однако, лучше было бы закодировать наш приватный ключ в SHA256, чтобы он не оказался излишне коротким, длиным, и т.п.

In [17]:
import hashlib

priv_hex_key = hashlib.sha256(str(int(priv_bin_key, 2)).encode()).hexdigest()

priv_hex_key

'12487f8ec0cc84700a73b1d7adf57f0e229a6967e3d3d7956c2b0544728a66ff'

Такой приватный ключ хранить неудобно. Bitcoin предоставляет данные, которые нужно вводить, хранить и т.п. в формате `Base58Check` (`WIF`, при кодировании приватного ключа). Ты об этом формате кодирования почитаешь в книге. Удобный, ничего сложного. 

Рассматривать его реализацию не будем, просто используем готовое решение из библиотеки `python-bitoinlib` (https://github.com/petertodd/python-bitcoinlib).

В этой библиотеке сырую функцию кодирование в `Base58Check` нужно вызывать так:

```
import bitcoin
bitcoin.base58.encode(байтовые-данные)
```

Но предварительно нам необходимо подготовить наш приватный ключ к конвертации в `Base58Check`. А именно`:` добавить байт `0x80` в начало шестнадцатиричной строки, чтобы `Base58Check` представление ключа начиналось с цифры 5 и добавить чек сумму в конец нашей шестнадцатиричной строки

1. Добавим байт `0x80` в начало нашего приватного ключа в шестнадцатеричном представлении:

In [18]:
priv_hex_key

'12487f8ec0cc84700a73b1d7adf57f0e229a6967e3d3d7956c2b0544728a66ff'

In [19]:
priv_hex_key = '80' + priv_hex_key

In [20]:
priv_hex_key

'8012487f8ec0cc84700a73b1d7adf57f0e229a6967e3d3d7956c2b0544728a66ff'

2. Вычислим и добавим чек сумму в конец. Чек сумма - это первые 4 байта вот такой радости: `sha256(sha256(наш приватный ключ))`

In [21]:
import codecs
import hashlib
import binascii

# из шестнадцатеричных данных в байты
priv_bytes_key = codecs.decode(priv_hex_key, 'hex_codec')
# два раза кодируем данные с помощью sha256 и берём первые 4 байта
check_sum_bytes = hashlib.sha256(hashlib.sha256(priv_bytes_key).digest()).digest()[:4]
# превращаем байтовое представление в 16-ричное
check_sum_hex = binascii.hexlify(check_sum_bytes)

check_sum_hex

b'c1e07714'

Добавим эти 4 байта в конец нашего приватного ключа в 16-ричном формате

In [22]:
priv_hex_key += check_sum_hex.decode()

priv_hex_key

'8012487f8ec0cc84700a73b1d7adf57f0e229a6967e3d3d7956c2b0544728a66ffc1e07714'

Далее передадим наш приватный ключ в 16-ричном формате с префиксом 0x80 и 4 контрольными байтами в функцию, которая вернёт нам эти же данные, но в формате `Base58Check`.

In [24]:
import codecs
import bitcoin.base58

priv_base58_key = bitcoin.base58.encode(codecs.decode(priv_hex_key, 'hex_codec'))

priv_base58_key

'5HxLbpmHNe3gbzAktfpmKeU2TLdMwLQMxaiSVPEXsCXU6a7cE9y'

Вот и всё. То, что это приватный ключ, выдаёт `5` в самом начале, которое появилось благодаря байту `0x80`, который мы добавили.

Из внешних библиотек я использовал только `python-bitconlib`. 

Устанока: `pip install python-bitcoinlib`

А `hashlib`, `codecs`, `binascii` доступны из коробки/